In [1]:
!pwd
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
'''
sys.path.append('/Users/suryaven/Documents/personal/sky/skyburst')
sys.path.append('/Users/suryaven/Documents/personal/sky/starburst/')
sys.path.append('/Users/suryaven/Documents/personal/sky/starburst/starburst/utils')
'''
# TODO: Generalize differences between sky laptop and personal latop paths
sys.path.append('/Users/surya/Documents/sky/skyburst/')
sys.path.append('/Users/surya/Documents/sky/starburst/')
sys.path.append('/Users/surya/Documents/sky/starburst/utils')

import starburst
import skyburst
from starburst import utils
import log_jobs
import submit_jobs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import time 
import copy 
from collections import OrderedDict 
import math
import heapq 
pd.set_option('display.max_columns', None)


/Users/surya/Documents/sky/starburst/starburst/utils


In [2]:
LOGS = {
    "05_01_2023": 1682925843,
    "05_02_2023": 1683099273,
    "05_03_2023": 1683132152,
    "05_03_2023_2": 1683173965,
    "05_04_2023_1": 1683184792, 
    "05_04_2023_2": 1683196845,
    "05_04_2023_3": 1683259533,
    "05_05_2023_4": 1683343491,
    "05_06_2023_4": 1683410859,
    "05_06_2023_5": 1683418719,
    "05_06_2023_6": 1683442154,
    "05_06_2023_7": 1683452077,
    "05_07_2023_1": 1683486242,
    "05_07_2023_2": 1683489564,
    "05_07_2023_3": 1683496107,
    "05_07_2023_4": 1683497538,
    "1683498468": 1683497538, 
    "1683498857": 1683498857,
    "1683499283": 1683499283,
    "1683528723": 1683528723, # Fixed JCT values
    "1683534589": 1683534589, # Large sweep with greater arrival rates
    "1683607638": 1683607638, # Modified arrival rate values to match simulator
    "1683625643": 1683625643, # Policy waits until cluster state is updated
    "1683627780": 1683627780, # Policy waits until cluster state and running pods are updated
    "1683680278": 1683680278, # Sweep with uniform wait (4 sec) and constant timeout (3 sec)
    "1683705558": 1683705558, # 30 second inter arrival time and 5 second job time
    "1683707012": 1683707012, # 30 second inter arrival time and 5 second job time (ROUND 2)
    "1683757260": 1683757260, # 30 second inter arrival time and 5 second job time (ROUND 3)
    "1683758384": 1683758384, # 5 second inter arrival time and 5 second job time (ROUND 3)
    "1683759834": 1683759834, # 5 second arrival 5 second job 1 second wait
}

logs = LOGS["1683759834"]

log_jobs.pull_vm_scheduler_logs(event_number=logs, force=True)
jobs_df = log_jobs.retrieve_df(event_number=logs, avoid_congestion=False)
jobs_df = jobs_df.assign(gpus_per_node=jobs_df['cpus_per_node'])

'<' not supported between instances of 'NoneType' and 'NoneType'


In [3]:
jobs_df

,allocated_gpus,arrival,arrival_rate,arrival_rate_sweep,batch_time,batch_time_sweep,cloud_cluster_nodes,cloud_cluster_nodes_sweep,cloud_cpu_per_node,cloud_cpu_per_node_sweep,cluster_size,cluster_size_sweep,cpu_dist,cpu_dist_sweep,cpu_sizes,cpu_sizes_sweep,cpus,cpus_per_node,cpus_per_node_sweep,fixed_values,gpu_dist,gpu_dist_sweep,gpu_sizes,gpu_sizes_sweep,idx,instance_type,mean_duration,mean_duration_sweep,memory_dict,memory_dict_sweep,memory_sizes,memory_sizes_sweep,node,node_index,num_gpus,random_seed,random_seed_sweep,runtime,start,submission_time,time_constrained,time_constrained_sweep,time_out,time_out_sweep,total_jobs,total_jobs_sweep,uniform_arrival,uniform_arrival_sweep,uniform_submission,uniform_submission_sweep,varying_values,wait_time,wait_time_sweep,wait_times,waiting_policy,waiting_policy_sweep,gpus_per_node
1,"[{1: []}, {0: []}, {3: []}, {0: []}, {1: []}, ...","[7.240361928939819, 7.240361928939819, 14.2403...",1,1,180,180,4,4,8,8,4,4,"[0.6, 0.25, 0.15]","[0.6, 0.25, 0.15]","[1, 2, 4]","[1, 2, 4]","[2, 2, 2, 2, 2, 2, 1, 1, 1, 4, 4, 4, 2, 2, 2, ...",8,8,"(batch_time, mean_duration, waiting_policy, cp...","[0, 0.2, 0.2, 0.2, 0.2, 0.2]","[0, 0.2, 0.2, 0.2, 0.2, 0.2]","[1, 2, 4, 8, 16, 32]","[1, 2, 4, 8, 16, 32]","[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, ...","[e2-standard-8, e2-standard-8, e2-standard-8, ...",5,5,"[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[100, 500, 1000, 50000]","[100, 500, 1000, 50000]","[gke-starburst-cpu-workloads-04fa1bcf-b5bt, gk...","[1, 0, 3, 0, 1, 3, 0, 1, 3, 2, 0, 2, 0, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0,"[7, 7, 8, 6, 6, 6, 20, 20, 20, 8, 8, 8, 3, 4, ...","[7.240361928939819, 7.240361928939819, 14.2403...","[0.0, 0.0, 0.0, 10.176762580871582, 10.1767625...",True,True,5,5,100,100,10,10,True,True,"(cpu_dist, wait_time, arrival_rate)",5,5,"[7.240361928939819, 7.240361928939819, 14.2403...",fifo_wait,fifo_wait,8


In [ ]:
jobs_df['arrival_mask'] = jobs_df['start'].apply(lambda x: [0 if not i else 1 for i in x])
jobs_df['onprem_mask'] = jobs_df['arrival_mask']

# CLIP WAITS

def cloud_wait_unclipped(row): 
    cloud_wait = [row['wait_times'][i] for i, n in enumerate(row['arrival_mask']) if n == 0]
    return cloud_wait
jobs_df['cloud_wait_unclipped'] = jobs_df.apply(cloud_wait_unclipped, axis=1)

def clipped_wait(row): 
    k8s_scheduling_waiting_constant = 1 #wait_time_2
    onprem_wait = [row['wait_times'][i] for i, n in enumerate(row['arrival_mask']) if n == 1]
    cloud_wait = [k8s_scheduling_waiting_constant + row['wait_time'] for i, n in enumerate(row['arrival_mask']) if n == 0]
    new_wait = onprem_wait + cloud_wait
    return new_wait

jobs_df['wait_times'] = jobs_df.apply(clipped_wait, axis=1)

def cloud_wait(row): 
    k8s_scheduling_waiting_constant = 1 #wait_time_2
    cloud_wait = [k8s_scheduling_waiting_constant + row['wait_time'] for i, n in enumerate(row['arrival_mask']) if n == 0]
    return cloud_wait
jobs_df['cloud_wait'] = jobs_df.apply(cloud_wait, axis=1)


def onprem_wait(row): 
    k8s_scheduling_waiting_constant = 1 #wait_time_2
    onprem_wait = [row['wait_times'][i] for i, n in enumerate(row['arrival_mask']) if n == 1]
    return onprem_wait

jobs_df['onprem_wait'] = jobs_df.apply(onprem_wait, axis=1)

# COMPUTE METRICS

jobs_df['avg_wait'] = jobs_df['wait_times'].apply(lambda x: sum(x)/len(x))
jobs_df['avg_runtime'] = jobs_df['runtime'].apply(lambda x: sum(x)/len(x))

def compute_total_time(row):
    total_time = [row['wait_times'][i] + row['runtime'][i] for i in range(len(row['wait_times']))]
    return total_time

jobs_df['total_time'] = jobs_df.apply(compute_total_time, axis=1)

def compute_completion_time(row):
    completion_time = [row['total_time'][i] + row['submission_time'][i] for i in range(len(row['wait_times']))]
    return completion_time

jobs_df['completion_time'] = jobs_df.apply(compute_completion_time, axis=1)
jobs_df['avg_jct'] = jobs_df['total_time'].apply(lambda x: sum(x)/len(x))

def compute_cluster_utilization(row):
    surface_area = [row['runtime'][i] * row['cpus'][i] for i in range(len(row['runtime']))]
    utilized_surface_area = sum(surface_area)
    total_surface_area = (max(row['completion_time']) - min(row['submission_time'])) * (row['cpus_per_node'] * row['cluster_size'])
    cluster_utilization = utilized_surface_area/total_surface_area
    return cluster_utilization

jobs_df['cluster_utilization'] = jobs_df.apply(compute_cluster_utilization, axis=1)

def compute_system_utilization(row):
    # TODO: Compute this value correctly
    return system_utilization

#jobs_df['cluster_utilization'] = jobs_df.apply(compute_cluster_utilization, axis=1)

GCP_PRICES = {
    "e2-medium": 0.038795,
    "e2-standard-8": 0.31036,
    "unknown": 0.038795,
}

def compute_total_cost(row):
    # TODO: Compute this value correctly
    # Get all cloud runtimes + submit 
    total_time = [row['runtime'][i] * i * GCP_PRICES[row['instance_type']] for i in range(len(row['onprem_mask']))]
    return system_utilization

jobs_df['total_cloud_cost'] = jobs_df.apply(compute_cluster_utilization, axis=1)


In [ ]:
jobs_df

In [ ]:
#jobs_df.filter(['arrival_rate', 'total_cloud_cost', 'wait_time', 'avg_jct'])
#jobs_df.filter(['waiting_policy', 'wait_time', 'wait_times', 'arrival', 'start', 'submission_time', 'arrival_rate', 'arrival_mask'])


In [ ]:
def wait_time_histograms(row):
    """Plots waiting time for onprem, cloud, and cloud clipped"""
    #import pdb; pdb.set_trace()
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12,3)) 
    
    axs[0].set_title("ONPREM " + "wait " + str(row['wait_time']) +  " arr " + str(row['arrival_rate']), fontsize=10)
    axs[0].hist(row['onprem_wait'])
    if row['onprem_wait'] != []:
        average = sum(row['onprem_wait'])/len(row['onprem_wait'])
        axs[0].axvline(average, color='r', linewidth=0.5)

    axs[1].set_title("CLOUD " + "wait " + str(row['wait_time']) +  " arr " + str(row['arrival_rate']), fontsize=10)
    axs[1].hist(row['cloud_wait'])
      
    axs[2].set_title("CLOUD UNCLIPPED " + "wait " + str(row['wait_time']) +  " arr " + str(row['arrival_rate']), fontsize=10)
    axs[2].hist(row['cloud_wait_unclipped'])
    
    plt.show()

jobs_df.apply(wait_time_histograms, axis=1)

In [ ]:
jobs_df

In [ ]:
def cpu_index_mapping(jobs=None, gpus_per_node=8):
    '''
    Implement greedly algorithm with heap to place cpu jobs to free cpu indices
    
    (1) Add all jobs to queue, then greedily assign indicies 
    (2) Have priority queue for each node with "Free indices" sorted by index number 
    (3) Iterate over all start times 

    TODO: Fix 1-off CPU index error 
    TODO: Don't let any job_idx values to be -1, and ensure it starts at node 1
    TODO: Verify allocated_nodes
    TODO: Determine how parsing changes between http_info and default values 
    TODO: Verify arrival value is accurate
    TODO: Create a list of times that include all arrival times and completion times in the same list in numerical order 
    '''
    
    GPUS_PER_NODE = gpus_per_node
    allocated_nodes = jobs['node_index']
    cpus = jobs['cpus']
    nodes = set(allocated_nodes)
    
    node_jobs ={}
    node_queues = {}
    for node in nodes:
        node_queues[node] = [i + 1 for i in range(GPUS_PER_NODE)]
        node_jobs[node] = []

    global_queue = [] # Queue sorted on end time -- earliest to latest end time
    job_id_to_index = {} 

    for i in range(len(jobs['arrival'])):
        #Remove values from queue
        #import pdb; pdb.set_trace()
        job_id = jobs['idx'][i]
        job_id_to_index[job_id] = i
        job_node = jobs['node_index'][i]
        job_cpu_size = jobs['cpus'][i]
        job_arrival = jobs['arrival'][i]
        job_runtime = jobs['runtime'][i]
        
        while global_queue and global_queue[0][0] <= job_arrival: 
            end_time, end_job_id = heapq.heappop(global_queue)
            released_index = job_id_to_index[end_job_id]
            for released_node in jobs['allocated_gpus'][released_index]: 
                released_cpus = jobs['allocated_gpus'][released_index][released_node]
                released_node_queue = node_queues[released_node]
                try: 
                    node_jobs[released_node].remove(end_job_id)
                except: 
                    #import pdb; pdb.set_trace()
                    print("Job Id not found")
                    print(end_job_id)
                    print(node_jobs[released_node])
                    continue 
                for cpu in released_cpus: 
                    heapq.heappush(released_node_queue, cpu)

        heapq.heappush(global_queue, (job_arrival + job_runtime, job_id))
        job_allocated_cpus = []
        node_queue = node_queues[job_node]
        node_jobs[job_node].append(job_id)
        
        try: 
            for j in range(job_cpu_size): 
                cpu_index = heapq.heappop(node_queue)
                job_allocated_cpus.append(cpu_index)
        except:
            print("not enough cpus to fit jobs")
            print(node_queue)
            print(job_allocated_cpus)
            
        jobs['allocated_gpus'][i] = {job_node: job_allocated_cpus}
        #import pdb; pdb.set_trace()
        
    return jobs

def generate_gantt_chart(row, gpus_per_node, ratio, scale):
    '''
    Create "threads index" that track CPU jobs running together
    #TODO: Modify function to plot CPU jobs --> number of jobs concurrently running may exceed cpu count
    #TODO: Plot color based on job start time and not job index
    #TODO: Determine why jobs dissapaear when strings labels are used for nodes
    #TODO: Label each row of jobs with the name of the node -- not just integers 
    '''
    #import pdb; pdb.set_trace()
    save=False; path=None; subplt=None; plt_index=None; tag=None; plot_sweep=False
    varying_values = row['varying_values']
    tag = ""
    
    for value in varying_values: 
        tag += str(row[value])
        tag += " | "
    
    
    fig, ax = plt.subplots()
    
    GPUS_PER_NODE = row['cpus_per_node']
    GPUS_PER_NODE = gpus_per_node
    row = cpu_index_mapping(row, GPUS_PER_NODE)
    #import pdb; pdb.set_trace()
    
    NUM_COLORS = len(row['idx']) + 5
    cm = plt.get_cmap('gist_rainbow')
    colors = [cm(1. * i / NUM_COLORS) for i in range(NUM_COLORS)]
    
    y_lim_min = 1000
    y_lim_max = -1000
    num_nodes = row['cluster_size'] + row['cloud_cluster_nodes']
    
    total_gpus = num_nodes * GPUS_PER_NODE #GPUs equivalent to CPUs
    segment_height_list = {}
    gpu_indices = {}
    node_name = ""
    try: 
        for j_idx in range(len(row['idx'])):
            allocated_gpus = row['allocated_gpus'][j_idx]
            segment = (row['arrival'][j_idx],
                        row['arrival'][j_idx] + row['runtime'][j_idx], j_idx)

            node_name = row['node'][j_idx]
            for node_idx in allocated_gpus.keys():
                for node_gpu_idx in allocated_gpus[node_idx]:
                    gpu_idx = total_gpus - (GPUS_PER_NODE * node_idx + node_gpu_idx)
                    gpu_indices[node_name] = [gpu_idx]
                    y_lim_min = min(y_lim_min, gpu_idx - 8)#if gpu_idx > 0 else gpu_index - 8)
                    y_lim_max = max(y_lim_max, gpu_idx + 8)
                    
                    plt.barh(gpu_idx,
                                width=row['runtime'][j_idx],
                                edgecolor='black',
                                height=1.0,
                                left=segment[0],
                                align='edge',
                                color=colors[row['idx'][j_idx]] if row['idx'][j_idx] < len(colors) else None,
                                alpha = 0.5)    
    except Exception as e: 
        print(e)

    for i in range(total_gpus + 1):
        multiplier = math.ceil(num_nodes / 32)
        if (i + 1) % GPUS_PER_NODE == 1:
            plt.axhline(y=i + 1, linewidth=0.75 / multiplier, color='black')
        else:
            pass

    max_arrival = max(row['arrival'])
    completions = [row['arrival'][i] + row['runtime'][i] for i in range(len(row['arrival']))]
    max_completion = max(completions)

    x_lim_max = max_completion
    last_job_time = max(row['completion_time'])    
    dim=(y_lim_min, y_lim_max, 0, last_job_time)
    bottom, top, left, right = dim
    plt.ylim(bottom=bottom, top=top)
    plt.xlim(left=left, right=right)
    plt.axvline(x=max_arrival, color='brown', linewidth=0.75)
    plt.xlabel('Time')
    plt.ylabel('Nodes ')
    plt.title(str(tag))

    gpu_labels = sorted([(v, k) for k, v in gpu_indices.items()])
    ticks = [label[0] for label in gpu_labels]
    ticks = np.array(ticks)
    ticks = ticks.flatten()
    labels = [label[1] for label in gpu_labels]
    labels = np.array(labels)
    labels = labels.flatten()
    plt.yticks(ticks)
    ax.set_yticklabels(labels)
    
    if save:
        if path: 
            plt.savefig(path)
            plt.close()
    else:
        plt.show()

        
gpus_per_node = 8
ratio = (1, 1)
scale = 1

jobs_df.apply(generate_gantt_chart, axis=1, args=(gpus_per_node, ratio, scale))

In [ ]:
def set_plotting_setting(ax):
    plt.rcParams.update({'font.size': 15})
    ax.grid(True, which='both')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    #ax.tick_params(bottom=False, left=False)
    ax.tick_params(bottom=True, left=False)
    ax.set_axisbelow(True)

label_dict = {
    'avg_jct': 'Avg. JCT (hr)',
    'cost_mult': '% Cost Savings\nover No Wait',
    'cost_diff': 'Cost Savings\nover No Wait',
    'cluster_size': 'Cluster Size (# Nodes)',
    'norm_system_utilization': 'System Utilization',
    'system_utilization': 'System Utilization',
    'cluster_utilization': 'Cluster Utilization',
    'total_cloud_cost': 'Cloud Cost',
    'arrival_rate': 'Arrival Rate',
}

legend_dict = {
    'constant': 'Constant',
    'linear_runtime': 'Runtime',
    'linear_cost': 'Cost',
    'zero': 'No Wait',
    'linear_runtime_filter_cpu': 'Runtime-Preempt-CPU'
}

def join_baseline(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(5*len(y_axis), 3.5))
    
    for k,v in df_filter.items():
        df = df[df[k]==v]
       
    baseline_df = df
    for k,v in baseline_filter.items():
        assert not isinstance(v, list)
        baseline_df = baseline_df[baseline_df[k]==v]
        #df = df[df[k]!=v]
    diff_df = pd.merge(df, baseline_df, left_on=x_axis,right_on=x_axis)
    return diff_df
    
def get_default(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
        
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(5*len(y_axis), 3.5))
    
    for k,v in df_filter.items():
        print(k)
        print(v)
        print(df[k])
        #print(df[k]==v)
        #df[k] = df[k].apply(lambda x: [0, 0, 0] if len(x) != 3 else x)
        #print(df[k].eq(v))
        
        
        mask = df[k].apply(lambda x: x == v)
        print(mask)
        if isinstance(v, list):
            df = df[mask]
            #df[k].eq(v)
            #df[k] = df[k].apply(lambda x: [0, 0, 0] if len(x) != 3 else x)
            #df = df[df[k].eq(v)]
            
        #df = df[df[k]==v]
        
    return df
    
def get_baseline(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(5*len(y_axis), 3.5))
    
    for k,v in df_filter.items():        
        mask = df[k].apply(lambda x: x == v)
        if isinstance(v, list):
            df = df[mask]
    
    baseline_df = df
    
    for k,v in baseline_filter.items():
        #assert not isinstance(v, list)
        #baseline_df = baseline_df[baseline_df[k]==v]
        mask = baseline_df[k].apply(lambda x: x == v)
        baseline_df = baseline_df[mask]
        baseline_df = df[mask]
        #print(mask)
        #if isinstance(v, list):
            #df[k].eq(v)
            #df[k] = df[k].apply(lambda x: [0, 0, 0] if len(x) != 3 else x)
            #df = df[df[k].eq(v)]
            
        #df = df[df[k]!=v]
    
    diff_df = pd.merge(df, baseline_df, left_on=x_axis,right_on=x_axis)
    
    if normalize_x_axis:
        if x_axis == 'cluster_size':
            # Hardcoded in Philly trace, precomputed ahead of time
            if df['dataset'].iloc[0] == 'philly':
                total_job_volume = 1155998.77277777
                job_makespan = 2559.3205555555555
            elif df['dataset'].iloc[0] == 'helios':
                total_job_volume = 1853756.1241666232
                job_makespan = 4651.911388888889
            diff_df['norm_system_utilization'] = total_job_volume/(job_makespan*diff_df['cluster_size']*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
        elif x_axis == 'arrival_rate':
            avg_job_volume_rate = diff_df['arrival_rate'] * np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            #print(avg_job_volume_rate)
            #print(df['cluster_size'].iloc[0])
            #print(df['gpus_per_node'].iloc[0])
            #print(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            diff_df['norm_system_utilization'] = avg_job_volume_rate/(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
        
    #return baseline_df
    return diff_df

def simulator_plotting_fn(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False,
                          fig_ratio=(5, 3.5)):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(fig_ratio[0]*len(y_axis), fig_ratio[1]))
    
    if len(y_axis) == 1: 
        if not isinstance(axs, list):
            axs = [axs]
    print(axs)
    #time.sleep(1000)
    
    '''
    for k,v in df_filter.items():
        if isinstance(v, list):
            df = df[df[k]==v]
    '''

    for k,v in df_filter.items():        
        mask = df[k].apply(lambda x: x == v)
        if isinstance(v, list):
            df = df[mask]
       
    baseline_df = df
    
    for k,v in baseline_filter.items():
        assert not isinstance(v, list)
        baseline_df = baseline_df[baseline_df[k]==v]
        #df = df[df[k]!=v]
        
    diff_df = pd.merge(df, baseline_df, left_on=x_axis,right_on=x_axis)
    
    if normalize_x_axis:
        if x_axis == 'cluster_size':
            # Hardcoded in Philly trace, precomputed ahead of time
            if df['dataset'].iloc[0] == 'philly':
                total_job_volume = 1155998.77277777
                job_makespan = 2559.3205555555555
            elif df['dataset'].iloc[0] == 'helios':
                total_job_volume = 1853756.1241666232
                job_makespan = 4651.911388888889
            diff_df['norm_system_utilization'] = total_job_volume/(job_makespan*diff_df['cluster_size']*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
        elif x_axis == 'arrival_rate':
            # Volume rate => runtime must be in either hours or seconds 
            '''
            Verification: 
            - Arrival_Rate = VERIFIED ~ error 
            - Num_gpus = Verified
            - Runtime = Verified 
            '''
            avg_job_volume_rate = diff_df['arrival_rate'] * np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            df['z_mean_runtime'] = np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            df['z_cluster_nodes'] = (df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            #print(avg_job_volume_rate)
            #print(df['cluster_size'].iloc[0])
            #print(df['gpus_per_node'].iloc[0])
            #print(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            diff_df['norm_system_utilization'] = avg_job_volume_rate/(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
    
    #print(df)
    
    def cost_multiplier(row):
        baseline_cost = row['total_cloud_cost_y']
        cost = row['total_cloud_cost_x']
        if baseline_cost == 0 and cost==0:
            return 0
        elif baseline_cost <=10000:
            # Small cloud cost for No wait
            # Savings over small cloud cost is negligible for organizations.
            return 0
        elif baseline_cost == 0 and cost>0:
            return 100
        return 100* (1 - (cost/baseline_cost))
    
    def cost_difference(row):
        baseline_cost = row['total_cloud_cost_y']
        cost = row['total_cloud_cost_x']
        return baseline_cost - cost
    
    diff_df['cost_mult'] = diff_df.apply(cost_multiplier, axis=1)
    diff_df['cost_diff'] = diff_df.apply(cost_difference, axis=1)
    groupby_values = [f'{g}_x' for g in groupby_values]
    mod_y_axis = [f'{y}_x' if y!='cost_mult' and y!='cost_diff' else y for y in y_axis]
    import itertools
    markers = itertools.cycle(('v', '^','.', 'o', '*',',', '+',)) 
    for idx, (label, grp) in enumerate(diff_df.groupby(groupby_values)):
        marker = next(markers)
#         if 'waiting_policy' in groupby_values[0]:
#             label = [legend_dict[label[0]]] + list(label[1:])
#             print(label)
        for ax_idx, ax in enumerate(axs):           
            grp.plot(x = x_axis, y = mod_y_axis[ax_idx],ylabel=label_dict[y_axis[ax_idx]], \
                     xlabel=label_dict[x_axis], marker=marker, ax = ax, label = label, legend=None)
    
    
    for ax in axs:
        set_plotting_setting(ax)
    #axs[1].set_xlim(right=36, left=20)
    lines, labels = ax.get_legend_handles_labels()
    fig.legend(lines, labels, ncol=len(labels), \
               bbox_to_anchor=(0, 0.92, 1, 0.2),loc='upper center')
    plt.tight_layout()
    plt.show()  

In [ ]:
jobs_df

In [ ]:
# TODO: Add spec to increase plot size

#dists = [[0.2, 0.4, 0.4], [0, 0.5, 0.5], [0, 0, 1]]
dists = [[0.6, 0.25, 0.15]]#[[0, 0.5, 0.5]]
scale = 4
for dist in dists:
    simulator_plotting_fn(jobs_df, x_axis='arrival_rate', \
                          y_axis=['avg_jct', 'cluster_utilization'],\
                          #['total_cloud_cost'], 
                          #['avg_jct'],\
                          #[avg_jct', 'total_cloud_cost'], \
                          df_filter={'cpu_dist': dist}, 
                          #baseline_filter= {'wait_time': 1.0}, \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0.0}, \
                          baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 2.5}, \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                          #groupby_values=['waiting_policy', 'wait_time'], normalize_x_axis=True, \
                          groupby_values=['wait_time'], normalize_x_axis=True, \
                          fig_ratio=(5*scale, 3.5*scale))
    


In [ ]:
default_df = get_default(jobs_df, x_axis='arrival_rate', y_axis=['avg_jct'], \
                           #df_filter={'cpu_dist': [0.2, 0.4, 0.4]}, \
                         #df_filter={'cpu_dist': [0.0, 0.5, 0.5]}, \
                         df_filter={'cpu_dist': [0.6, 0.25, 0.15]}, \
                          #df_filter={'wait_time': 1.0}, \
                      #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0.0}, \
                         baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0}, \
                           groupby_values=['waiting_policy'], normalize_x_axis=True)


baseline_df = get_baseline(jobs_df, x_axis='arrival_rate', y_axis=['avg_jct'], \
                           #df_filter={'cpu_dist': [0.2, 0.4, 0.4]}, \
                           #df_filter={'cpu_dist': [0.0, 0.5, 0.5]}, \
                           df_filter={'cpu_dist': [0.6, 0.25, 0.15]},
                      #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                           baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0}, \
                           groupby_values=['waiting_policy'], normalize_x_axis=True)


merged_df = join_baseline(jobs_df, x_axis='arrival_rate', y_axis=['avg_jct', 'total_cloud_cost'], \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                          baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0}, \
                      groupby_values=['waiting_policy'], normalize_x_axis=True)
default_df, baseline_df, merged_df

In [ ]:
#jobs_df

In [ ]:
hyp = copy.deepcopy(sampled_jobs.DEFAULT_HYPERPARAMETERS)

hyp['arrival_rate'] = 10
hyp['batch_time'] = 600
hyp['total_jobs'] = 5
hyp['mean_duration'] = 60
arrived_jobs, arrivals = sampled_jobs.generate_jobs(hyp)

#print(arrived_jobs)
#print(arrivals)

In [ ]:
## HYPERPARAMETER
'''
Vary arrival rate
    - CPU jobs for Philly/Helios
        
- Mimic GPU jobs ~ with CPU
    - vary # jobs per minute 
    - test contant waiting for baseline
    - 0(Baseline), 0.25, 0.5, 1 ratio of runtime for waiting policy 
        - 
    
- Plot Cluster Utilization 
- Plot Cloud Cost 
'''